In [5]:
# install.packages('devtools')
install.packages('prophet')

also installing the dependencies 'checkmate', 'matrixStats', 'inline', 'gridExtra', 'loo', 'V8', 'BH', 'dygraphs', 'extraDistr', 'RcppParallel', 'rstan', 'rstantools', 'StanHeaders', 'RcppEigen'




  There are binary versions available but the source versions are later:
            binary source needs_compilation
matrixStats 0.58.0 0.61.0              TRUE
inline      0.3.17 0.3.19             FALSE
V8           3.4.2  3.6.0              TRUE

  Binaries will be installed
package 'checkmate' successfully unpacked and MD5 sums checked
package 'matrixStats' successfully unpacked and MD5 sums checked
package 'gridExtra' successfully unpacked and MD5 sums checked
package 'loo' successfully unpacked and MD5 sums checked
package 'V8' successfully unpacked and MD5 sums checked
package 'BH' successfully unpacked and MD5 sums checked
package 'dygraphs' successfully unpacked and MD5 sums checked
package 'extraDistr' successfully unpacked and MD5 sums checked
package 'RcppParallel' successfully unpacked and MD5 sums checked
package 'rstan' successfully unpacked and MD5 sums checked
package 'rstantools' successfully unpacked and MD5 sums checked
package 'StanHeaders' successfully unpacked a

installing the source package 'inline'



In [1]:
library(fpp3)
library(repr)
library(IRdisplay)
library(evaluate)
library(crayon)
library(pbdZMQ)
# library(devtools)
library(uuid)
library(digest)
library(ggplot2)
library(gtable)
library(grid)
# library(prophet)
# library(forecast)

Warning message:
"package 'fpp3' was built under R version 3.6.3"-- Attaching packages ------------------------------------------------------------------------------------ fpp3 0.4.0 --
v tibble      3.1.1     v tsibble     1.1.0
v dplyr       1.0.6     v tsibbledata 0.3.0
v tidyr       1.1.3     v feasts      0.2.2
v lubridate   1.7.4     v fable       0.3.0
v ggplot2     3.3.5     
Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'tsibbledata' was built under R version 3.6.3"Warning message:
"package 'fabletools' was built under R version 3.6.3"Warning message:
"package 'fable' was built under R version 3.6.3"-- Conflicts ----------------------------------------------------------------------------------------- fpp3_conflicts --
x lubridate::date()       masks base::date()
x dplyr::filter()         masks stats::

In [2]:
#Round function we will use later or not, ignore for now
round2 = function(x, n) {
  posneg = sign(x)
  z = abs(x)*10^n
  z = z + 0.5 + sqrt(.Machine$double.eps)
  z = trunc(z)
  z = z/10^n
  z*posneg
}

### Data Gathering

In [3]:
train <- read.csv("sales_train_validation_afcs2021.csv")
test <- read.csv("test.csv")
calendar <- read.csv("calendar.csv")

### Data Preprocessing

In [4]:
train = as_tibble(cbind(nms = names(train), t(train)))
test = as_tibble(cbind(nms = names(test), t(test)))

Warning message:
"The `x` argument of `as_tibble.matrix()` must have unique column names if `.name_repair` is omitted as of tibble 2.0.0.
Using compatibility `.name_repair`.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated."

In [5]:
test = test %>%  slice(2 : n()) %>% tibble::rowid_to_column("Day") %>% 
  select(-nms)%>%  mutate(Day = Day +1913) %>% 
  as_tsibble(index = Day) %>% mutate_each(funs(as.numeric), starts_with("V")) 

train = train %>%  slice(2 : n()) %>% tibble::rowid_to_column("Day") %>% 
  select(-nms)%>% 
  as_tsibble(index = Day) %>% mutate_each(funs(as.numeric), starts_with("V"))

Warning message:
"`funs()` was deprecated in dplyr 0.8.0.
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated."Warning message:
"`mutate_each_()` was deprecated in dplyr 0.7.0.
Please use `across()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_lifecycle_warnings()` to see where this warning was generated."

In [6]:
calendar

date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA
1/29/2011,11101,Saturday,1,1,2011,d_1,,,,,0
1/30/2011,11101,Sunday,2,1,2011,d_2,,,,,0
1/31/2011,11101,Monday,3,1,2011,d_3,,,,,0
2/1/2011,11101,Tuesday,4,2,2011,d_4,,,,,1
2/2/2011,11101,Wednesday,5,2,2011,d_5,,,,,1
2/3/2011,11101,Thursday,6,2,2011,d_6,,,,,1
2/4/2011,11101,Friday,7,2,2011,d_7,,,,,1
2/5/2011,11102,Saturday,1,2,2011,d_8,,,,,1
2/6/2011,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,,,1
2/7/2011,11102,Monday,3,2,2011,d_10,,,,,1


In [7]:
# Preprocess the whole calendar
# 1. Put number instead of event_type_1
# 2. Put 0 for Sunday anf 1 for any other day

calendar = calendar %>%  mutate(Event = case_when(
    event_type_1 == "Sporting" ~ 1,
    event_type_1 == "Cultural" ~ 2,
    event_type_1 == "National" ~ 3,
    event_type_1 == "Religious" ~ 4,
    TRUE ~ 0
  ),Day_type = case_when(
    weekday == "Sunday" ~ 0,
    TRUE~ 1
  )) 

In [8]:
calendar

date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
1/29/2011,11101,Saturday,1,1,2011,d_1,,,,,0,0,1
1/30/2011,11101,Sunday,2,1,2011,d_2,,,,,0,0,0
1/31/2011,11101,Monday,3,1,2011,d_3,,,,,0,0,1
2/1/2011,11101,Tuesday,4,2,2011,d_4,,,,,1,0,1
2/2/2011,11101,Wednesday,5,2,2011,d_5,,,,,1,0,1
2/3/2011,11101,Thursday,6,2,2011,d_6,,,,,1,0,1
2/4/2011,11101,Friday,7,2,2011,d_7,,,,,1,0,1
2/5/2011,11102,Saturday,1,2,2011,d_8,,,,,1,0,1
2/6/2011,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,,,1,1,0
2/7/2011,11102,Monday,3,2,2011,d_10,,,,,1,0,1


In [9]:
calendarTest = calendar %>% slice(n()-27:0)%>% tibble::rowid_to_column("Day") %>%  mutate(Day = Day +1913)
calendar = calendar %>% slice(0 : 27-n())
calendarTrain = calendar %>% slice(0 : 27-n()) %>% tibble::rowid_to_column("Day") 

In [10]:
# Merge sets test,train and whole with their calendars

test =as_tsibble(merge(test, calendarTest, by.x = "Day", by.y= "Day"), index= "Day") 
test  = rename(test, Date = date)
test = test %>% mutate(Date = as.Date(Date, format = "%m/%d/%Y"))

train =as_tsibble(merge(train, calendarTrain, by.x = "Day", by.y= "Day"), index= "Day") 
train  = rename(train, Date = date)
train = train %>% mutate(Date = as.Date(Date, format = "%m/%d/%Y"))

In [11]:
train

Day,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
1,0,0,0,0,0,0,27,0,0,...,1,2011,d_1,,,,,0,0,1
2,2,0,0,0,1,0,27,0,0,...,1,2011,d_2,,,,,0,0,0
3,1,0,0,0,1,0,8,0,0,...,1,2011,d_3,,,,,0,0,1
4,3,0,0,0,0,0,0,0,0,...,2,2011,d_4,,,,,1,0,1
5,0,0,0,0,0,0,0,0,0,...,2,2011,d_5,,,,,1,0,1
6,0,0,0,0,1,0,11,0,0,...,2,2011,d_6,,,,,1,0,1
7,1,0,0,0,2,0,8,0,0,...,2,2011,d_7,,,,,1,0,1
8,0,0,0,0,0,0,33,0,0,...,2,2011,d_8,,,,,1,0,1
9,2,0,0,0,0,0,16,0,0,...,2,2011,d_9,SuperBowl,Sporting,,,1,1,0
10,1,0,0,0,1,0,14,0,0,...,2,2011,d_10,,,,,1,0,1


In [12]:
# Bind test and train to create full set
whole = bind_rows(train,test)

In [13]:
whole

Day,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
1,0,0,0,0,0,0,27,0,0,...,1,2011,d_1,,,,,0,0,1
2,2,0,0,0,1,0,27,0,0,...,1,2011,d_2,,,,,0,0,0
3,1,0,0,0,1,0,8,0,0,...,1,2011,d_3,,,,,0,0,1
4,3,0,0,0,0,0,0,0,0,...,2,2011,d_4,,,,,1,0,1
5,0,0,0,0,0,0,0,0,0,...,2,2011,d_5,,,,,1,0,1
6,0,0,0,0,1,0,11,0,0,...,2,2011,d_6,,,,,1,0,1
7,1,0,0,0,2,0,8,0,0,...,2,2011,d_7,,,,,1,0,1
8,0,0,0,0,0,0,33,0,0,...,2,2011,d_8,,,,,1,0,1
9,2,0,0,0,0,0,16,0,0,...,2,2011,d_9,SuperBowl,Sporting,,,1,1,0
10,1,0,0,0,1,0,14,0,0,...,2,2011,d_10,,,,,1,0,1


### Models

In [20]:
# train
# test
# calendar

write.csv(train, 'train_preprocessed.csv')
write.csv(test, 'test_preprocessed.csv')
write.csv(calendar, 'calendar_preprocessed.csv')

In [14]:
test

Day,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
1914,1,0,0,0,1,0,8,0,1,...,5,2016,d_1942,,,,,0,0,1
1915,0,0,0,0,1,1,0,0,0,...,5,2016,d_1943,,,,,0,0,1
1916,0,0,1,0,2,1,3,0,0,...,5,2016,d_1944,,,,,0,0,1
1917,1,0,0,0,0,0,0,0,1,...,5,2016,d_1945,,,,,0,0,1
1918,0,0,0,1,2,1,15,0,2,...,5,2016,d_1946,,,,,0,0,1
1919,1,0,0,0,0,1,0,0,0,...,5,2016,d_1947,,,,,0,0,1
1920,0,0,0,1,2,3,12,0,2,...,5,2016,d_1948,,,,,0,0,0
1921,0,0,0,0,1,1,6,0,0,...,5,2016,d_1949,MemorialDay,National,,,0,3,1
1922,0,0,1,0,4,0,14,0,0,...,5,2016,d_1950,,,,,0,0,1
1923,0,2,1,0,0,2,17,0,0,...,6,2016,d_1951,,,,,1,0,1


In [15]:
# 1. ARIMA
train1 = train %>% select(V2,Day,month,Event,year)

test1 = test %>% select(V2,Day,month,Event,year)

whole1 = whole %>% select(V2,Date,month,Event,year)
          
fit1 = train1 %>% model("arima" =ARIMA(V2~ Day + month +Event  + year + 1  +pdq(2,0,4)))

report(fit1)
forecast(fit1,test1) %>% accuracy(whole1)

Series: V2 
Model: LM w/ ARIMA(2,0,4) errors 

Coefficients:
         ar1     ar2      ma1      ma2      ma3      ma4      Day   month
      0.2564  0.7227  -0.1968  -0.6638  -0.0180  -0.0306  -0.0008  0.0142
s.e.  1.0652  1.0522   1.0665   0.9905   0.0285   0.0426   0.0019  0.0582
        Event    year  intercept
      -0.0235  0.2414  -484.8805
s.e.   0.0189  0.6912  1390.1758

sigma^2 estimated as 0.4909:  log likelihood=-2028.49
AIC=4080.99   AICc=4081.15   BIC=4147.67


.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
arima,Test,-0.05348963,0.81681,0.5893792,-Inf,Inf,1.227552,0.8508677,-0.001571253


In [16]:
# OVERALL: Arima for all

df_rmse_arima =data.frame()

#Loop in all columns
#for(i in 2:ncol(whole)){

#Loop only in first 200 columns
for(i in 2:200){
  print(i)
  
  #create column name
  V_value <- paste("V", i, sep = "")
  
  train1 = train %>% select(V_value,Date,month,Event,year)
  test1 = test %>% select(V_value,Date,month,Event,year)
  whole11 = whole %>% select(V_value,Date,month,Event,year)
  
  #select the column of the train set for the fit of the model
  a <-assign(V_value, train1[1])
  
  fit111 = train1 %>% model("arima" =ARIMA(a ~ Day + month +Event  + year + 1  +pdq(2,0,4)))
  
  fore1  <- forecast(fit111,test1)
  
  #drop the column with the same name but wrong data
  fore1_cleaned <- fore1[-c(5)]
  #rename column so it would be the same for the accuracy function
  names(fore1_cleaned)[3] <- V_value
  #convert to fable fo the accuracy function
  fbl <- as_fable(fore1_cleaned, response = V_value, distribution =V_value)
  
  current_accuracy <- accuracy(fbl,whole11)
  
  new_df_rmse_arima<-data.frame(V_value,current_accuracy[4])
  
  df_rmse_arima=rbind(df_rmse_arima, new_df_rmse_arima)
}

[1] 2


Note: Using an external vector in selections is ambiguous.
i Use `all_of(V_value)` instead of `V_value` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.


[1] 3


Warning message in wrap_arima(y, order = c(p, d, q), seasonal = list(order = c(P, :
"possible convergence problem: optim gave code = 1"

[1] 4
[1] 5
[1] 6


...

Warning message in wrap_arima(y, order = c(p, d, q), seasonal = list(order = c(P, :
"possible convergence problem: optim gave code = 1"

[1] 198
[1] 199


Warning message in wrap_arima(y, order = c(p, d, q), seasonal = list(order = c(P, :
"possible convergence problem: optim gave code = 1"

[1] 200


Warning message in wrap_arima(y, order = c(p, d, q), seasonal = list(order = c(P, :
"possible convergence problem: optim gave code = 1"

In [17]:
df_rmse_arima

V_value,RMSE
V2,0.81681003
V3,1.21118138
V4,0.88219477
V5,0.38806262
V6,1.46672616
V7,1.55302345
V8,NaN
V9,9.31597925
V10,1.08726819
V11,0.96632256


In [18]:
# remove NaNs
df_rmse_arima[is.na(df_rmse_arima)] <- 1

df_rmse_arima
unique(df_rmse_arima$RMSE)
mean(unique(df_rmse_arima$RMSE))

V_value,RMSE
V2,0.81681003
V3,1.21118138
V4,0.88219477
V5,0.38806262
V6,1.46672616
V7,1.55302345
V8,1.00000000
V9,9.31597925
V10,1.08726819
V11,0.96632256


[1]  0.81681003  1.21118138  0.88219477  0.38806262  1.46672616  1.55302345
  [7]  1.00000000  9.31597925  1.08726819  0.96632256  0.82127019  0.97926921
 [13]  0.92321049  3.30005492  0.97985294  0.41244823  1.47200263  0.79097395
 [19]  2.15246661  1.75437454  0.09977306  1.02936081  1.85280039  1.66616403
 [25]  1.05297459  0.66626724  0.87052825  1.95875106  7.99341217  0.78197634
 [31]  1.15519903  0.74405875  0.94690014  0.50581927  1.20969344  0.30973449
 [37]  2.20151142  0.52169749  1.81347518  2.41594438  0.53627449  0.68759430
 [43]  0.61332836  0.57068942  2.61907497  1.20599098  0.91823801  0.69691564
 [49]  2.16309510  0.77617128  0.97690264  0.96601080  0.53688787  0.27379279
 [55]  1.39642945  2.14054889  0.66753057  0.44591795  0.86957759  6.11370856
 [61]  0.79530373  1.81709067  1.10056507  0.60570756  1.73260091 13.25805923
 [67]  2.61036439  1.86774379  0.87058807  0.61364660  3.67657868  2.45088478
 [73]  0.54576369  1.74146064  3.41541821  1.54657122  0.22282750  1.23254383
 [79]  0.95505233  1.03366028  6.26509100  2.42135168  0.49084735  1.60462420
 [85] 25.68876916  0.75245220  0.73396155  2.21526609  1.13997180  0.54937434
 [91]  1.51565976  3.64721685  1.15802429  2.74911073  0.68315193  0.87665179
 [97]  0.79665906  2.00652105  1.53260424  1.71187046  8.16807096  0.86584215
[103]  0.63801197  1.84536532  1.76298870  0.93088331  1.17643457  1.53736422
[109]  5.88419166  1.59277777  0.79901793 12.45824470  1.96996321  1.18905968
[115]  0.60864655  3.21997407  1.29539755  2.18006758  1.45410335  0.87220717
[121]  1.13679628  1.20759118  1.31289925  1.61588842  3.56927232  1.57464761
[127]  4.06034171  7.42550193  0.68671779  2.44239252  1.11063391  0.64318737
[133]  0.52685564  1.06157645  1.61729332  0.44230850  2.78183596  3.81690669
[139]  1.21308937  1.48740403  7.72455692  3.07667431  2.36915440  1.33929385
[145]  3.45526391  0.41166426  1.06183299  0.52652233  0.76805171  0.45470265
[151]  0.38486467  0.19101688  0.51278884  0.43810545  1.82744510  1.35122938
[157]  1.35452869  1.81320827  1.05349378  0.18656726  1.84869340  0.53795395
[163]  1.65736406  0.41945122  1.69080723  0.49573210  2.76467641  0.47519712
[169]  0.38428212  1.10238628  1.74317352  1.18288293  3.30035276  0.62492938
[175]  1.77950480  0.61361911  0.72191125  2.04577408  1.32495270  0.91134749
[181]  0.31793882  0.67700178  0.52680640  1.37382367  0.53080160  0.83647713
[187]  0.60616000  1.63260223

[1] 1.811091

In [21]:
train

Day,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,Event,Day_type
1,0,0,0,0,0,0,27,0,0,...,1,2011,d_1,,,,,0,0,1
2,2,0,0,0,1,0,27,0,0,...,1,2011,d_2,,,,,0,0,0
3,1,0,0,0,1,0,8,0,0,...,1,2011,d_3,,,,,0,0,1
4,3,0,0,0,0,0,0,0,0,...,2,2011,d_4,,,,,1,0,1
5,0,0,0,0,0,0,0,0,0,...,2,2011,d_5,,,,,1,0,1
6,0,0,0,0,1,0,11,0,0,...,2,2011,d_6,,,,,1,0,1
7,1,0,0,0,2,0,8,0,0,...,2,2011,d_7,,,,,1,0,1
8,0,0,0,0,0,0,33,0,0,...,2,2011,d_8,,,,,1,0,1
9,2,0,0,0,0,0,16,0,0,...,2,2011,d_9,SuperBowl,Sporting,,,1,1,0
10,1,0,0,0,1,0,14,0,0,...,2,2011,d_10,,,,,1,0,1


In [23]:
# Place to try more advanced methods(Prophet,Fourrier,NeuralNet)

# 2. STL Decompostion with seasonal adjusted data, 0.79 RMSE
# Best Results so far

try = ts(train$V2)

try = as_tsibble(try) %>% rename(V2 = value)
dcmp =  decomposition_model(
  STL(V2 ~ season(period = 7)  , robust = TRUE),
  ETS(season_adjust))



fit = try %>% model(dcmp)
fc=forecast(fit,test1)
fc %>%accuracy(whole1)

.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
dcmp,Test,-0.13726,0.790999,0.511183,-Inf,Inf,1.064686,0.8239805,0.1611853


In [24]:
# 3. Fourier methods are here, cant make them work
try = train %>% select(V2,Date,month,Event,year,Day_type)
whole1 = whole %>% select(V2,Date,month,Event,year,Day_type)
test1 = test %>% select(V2,Date,month,Event,year,Day_type)


try = as_tsibble(try, index= "Date") %>%  select(-Day)
whole1 = as_tsibble(whole1, index= "Date") %>%  select(-Day)
test1 = as_tsibble(test1, index= "Date") %>%  select(-Day)
FIT = try %>% model(
                   try =ARIMA( V2 ~ Date +month +Event  + 1 + PDQ(0,0,0) +pdq(3,0,2) )
                  )

fc= forecast(FIT, test1)
accuracy(fc, whole1)

Warning message in wrap_arima(y, order = c(p, d, q), seasonal = list(order = c(P, :
"possible convergence problem: optim gave code = 1"Warning message in sqrt(diag(best$var.coef)):
"NaNs produced"

.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
try,Test,-0.03267654,0.8187893,0.5858904,-Inf,Inf,1.235295,0.8589337,0.0002003573
